<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/MarkupLM/Fine_tune_MarkupLMForTokenClassification_on_a_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up environment

First, we install 🤗 Transformers.

We also install 🤗 Evaluate and Seqeval, for computing metrics like F1, recall and precision.

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
torch.cuda.device_count()

1

## Prepare dataset

Next, let's load a toy dataset which we'll use to fine-tune MarkupLM on.

The goal for the model is to label nodes of HTML strings with the appropriate class.

In [3]:
import numpy as np

def sample_websites(dataset, k, seed):
    all_websites = list(dataset.keys())
    training_websites = np.random.choice(all_websites, size=k, replace=False)
    print(training_websites)

    train = []
    valid = []
    for website in all_websites:
        if website in training_websites:
            train += dataset[website]
        else:
            valid += dataset[website]
    
    return train, valid

In [4]:
import json

with open("/home/savkin/vera/all_camera_datasets_merged.json", "r") as file:
    data = json.load(file)

train_data, valid_data = sample_websites(data, k=5, seed=0)

['jr' 'pcnation' 'ecost' 'amazon' 'beachaudio']


In [5]:
print(train_data[0])

{'nodes': [['Canon 3506b001 12.1 Megapixel Waterproof Powershot D10 (3508b001)', 'var runInitFB = true;', '[if lte IE 7]>\n<link href="/site/files/min_iexplorer.css?v=3.4.1" rel="stylesheet" type="text/css" />\n<![endif]', 'Home', '|', 'Order Status', '|', 'Help Center', '|', 'FAQ', '|', 'Return Request', '|', 'Log In', 'SEARCH', 'Entire Site', 'audio', 'video', 'dvd', 'photography', 'communications', 'gaming', 'car', 'computers', 'appliances', 'music & dj', 'blank media', 'office', 'for', 'SHOP BY BRAND', 'audio', 'video', 'dvd', 'photography', 'communications', 'gaming', 'car', 'computers', 'appliances', 'music & dj', 'blank media', 'office', 'More...', 'Home', '»', 'Photography', '»', 'Digital Cameras', '»', '3508B001', '[productwiki]', '[/productwiki]', 'Power Reviews p:1', 'var pr_page_id =\'271456\';\n    var pr_zip_location=\'/powerreviews\';\n    var pr_write_review=\'/review_submission.php?pageId=271456&products_id=271456\';\n\tvar pr_write_answer=\'/question_submission.php?pa

## Create PyTorch Dataset

Next, we'll create a regular PyTorch dataset. Each item of the dataset is an HTML string, encoded using MarkupLMProcessor. Note that we initialize the processor with parse_html = False, as we have already parsed the HTML ourselves and we're providing the nodes, xpaths and node labels.

Note that by default, the processor will only label the first token of a given node and label the remaining tokens with -100. you can change this by setting the `only_label_first_subword` attribute of the processor's tokenizer to `False`.

In [6]:
from torch.utils.data import Dataset

max_length=384

class MarkupLMDataset(Dataset):
    """Dataset for token classification with MarkupLM."""

    def __init__(self, data, processor=None, max_length=max_length):
        self.data = data
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # first, get nodes, xpaths and node labels
        item = self.data[idx]
        nodes, xpaths, node_labels = item['nodes'], item['xpaths'], item['node_labels']

        # provide to processor
        encoding = self.processor(nodes=nodes, 
                                  xpaths=xpaths, 
                                  node_labels=node_labels, 
                                  padding=True, 
                                  truncation=True,
                                  max_length=self.max_length, 
                                  return_tensors="pt")

        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}

        return encoding

In [7]:
from transformers import MarkupLMProcessor

processor = MarkupLMProcessor.from_pretrained("microsoft/markuplm-base")
processor.parse_html = False

train_dataset = MarkupLMDataset(data=train_data, processor=processor, max_length=max_length)
valid_dataset = MarkupLMDataset(data=valid_data, processor=processor, max_length=max_length)

/home/savkin/vera/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create PyTorch Dataloaders

The next step is to create a PyTorch DataLoader, which allows us to get batches from the dataset.

In [8]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
eval_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

## Define model

We define the model here, which is a MarkupLM-base Transformer, with a token classifier head on top. The token classifier will have randomly initialized weights, while the base Transformer has pre-trained weights.



In [9]:
from transformers import MarkupLMForTokenClassification

id2label = {0: "model", 1: "price", 2: "manufacturer", 3: "other"}
label2id = {label:id for id, label in id2label.items()}

model = MarkupLMForTokenClassification.from_pretrained("microsoft/markuplm-base", id2label=id2label, label2id=label2id)

Some weights of MarkupLMForTokenClassification were not initialized from the model checkpoint at microsoft/markuplm-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We also create a label_list, where each tag starts with a B (as seqeval expects the labels to be in IOB format).

In [10]:
label_list = ["B-" + x for x in list(id2label.values())]
label_list

['B-model', 'B-price', 'B-manufacturer', 'B-other']

We also define metric calculations (as we'd like to know the F1 score etc. during training). We'll use 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) for that, which is a library containing many tools for evaluating ML models.

In [11]:
import evaluate

# Metric
metric = evaluate.load("seqeval")

def get_labels(predictions, references):
    # Transform predictions and references tensos to numpy arrays
    if device.type == "cpu":
        y_pred = predictions.detach().clone().numpy()
        y_true = references.detach().clone().numpy()
    else:
        y_pred = predictions.detach().cpu().clone().numpy()
        y_true = references.detach().cpu().clone().numpy()


    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    return true_predictions, true_labels

def compute_metrics(return_entity_level_metrics=True):
    results = metric.compute()
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

## Train

Alright, let's train! Here we're training the model in native PyTorch, but of course you could also opt for things like 🤗 Accelerate, 🤗 Trainer, PyTorch Lightning,...

In [12]:
import torch
from torch.optim import AdamW
from tqdm.auto import tqdm

optimizer = AdamW(model.parameters(), lr=2e-5)

model.to(device)

model.train()
for epoch in range(5):  # loop over the dataset multiple times
    for batch in tqdm(train_dataloader):
        # get the inputs;
        inputs = {k:v.to(device) for k,v in batch.items()}

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(**inputs)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        print("Loss:", loss.item())

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        preds, refs = get_labels(predictions, labels)
        metric.add_batch(
            predictions=preds,
            references=refs,
        )

    eval_metric = compute_metrics()
    print(f"Epoch {epoch}:", eval_metric)

  2%|▏         | 1/56 [00:05<05:12,  5.69s/it]

Loss: 1.4233579635620117


  4%|▎         | 2/56 [00:09<04:10,  4.63s/it]

Loss: 0.9820742607116699


  5%|▌         | 3/56 [00:14<04:08,  4.69s/it]

...

 98%|█████████▊| 55/56 [04:59<00:06,  6.12s/it]

Loss: 0.0007193561759777367


100%|██████████| 56/56 [05:01<00:00,  5.39s/it]

Loss: 0.000334622134687379


Epoch 4: {'model_precision': 0.0, 'model_recall': 0.0, 'model_f1': 0.0, 'model_number': 17, 'other_precision': 0.9995317836289523, 'other_recall': 1.0, 'other_f1': 0.999765836995, 'other_number': 36291, 'overall_precision': 0.9995317836289523, 'overall_recall': 0.9995317836289523, 'overall_f1': 0.9995317836289523, 'overall_accuracy': 0.9995317836289523}


In [ ]:
metric = evaluate.load("seqeval")

model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)


    predictions = outputs.logits.argmax(dim=-1)
    labels = batch["labels"]

    preds, refs = get_labels(predictions, labels)

    metric.add_batch(
        predictions=preds,
        references=refs,
    )


eval_metrics = metric.compute()

In [14]:
eval_metrics

{'manufacturer': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 100},
 'model': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 77},
 'other': {'precision': 0.9946530525934205,
  'recall': 1.0,
  'f1': 0.9973193596752942,
  'number': 32926},
 'overall_precision': 0.9946530525934205,
 'overall_recall': 0.9946530525934205,
 'overall_f1': 0.9946530525934205,
 'overall_accuracy': 0.9946530525934205}

In [ ]:
# res = {}

# for x in all_refs:
#     for y in x:
#         y = int(y)
#         if y in res:
#             res[y] += 1
#         else:
#             res[y] = 1
# print(res)

In [ ]:
# from collections import Counter

# count = {
#     0: 0,
#     1: 0,
#     2: 0,
#     3: 0
# }

# for example in valid_data:
#     example_cnt = Counter(example["node_labels"][0])
#     # print(example_cnt)
#     for key in count.keys():
#         count[key] += example_cnt[key]

# print(count)

## Inference

Let's try out the model on a new web page for which we have the nodes and xpaths. Here we'll just use one of our training set.

In [ ]:
nodes = data[0]['nodes']
xpaths = data[0]['xpaths']
node_labels = data[0]['node_labels']
print("Nodes:", nodes)
print("Xpaths:", xpaths)

We'll prepare the example for the model using the processor. Note that we're passing `return_offsets_mapping=True`, as the offsets allow us to determine which tokens are at the start of a given word at which aren't.

In [ ]:
# prepare for model
# note that you don't need to prepare node_labels, we just have them available here so we'll compare to the ground truth
# print(processor.max_length)
encoding = processor(nodes=nodes, xpaths=xpaths, node_labels=node_labels, truncation=True, max_length=max_length, return_offsets_mapping=True, return_tensors="pt").to(device)
for k,v in encoding.items():
  print(k,v.shape)

Let's perform a forward pass:

In [ ]:
# we don't need the offset mapping and labels for the forward pass
offset_mapping = encoding.pop("offset_mapping")
labels = encoding.pop("labels")

# forward pass
with torch.no_grad():
  outputs = model(**encoding)

The model outputs logits of shape (batch_size, seq_len, num_labels). We just take the highest logit (score) per token as prediction:

In [ ]:
predictions = outputs.logits.argmax(-1)
print(predictions)

The model makes predictions at the token level, however we're only interested in the predicted label for the first token of each node.

This can be achieved by accessing the word_ids (to know whether or not the token is a special token or not) and the offset_mapping (to know whether or not the token is the first of a particular node).

In [ ]:
results = {"Node": [], "Predicted": [], "Ground truth": []}

for pred_id, word_id, offset, label_id in zip(predictions[0].tolist(), encoding.word_ids(0), offset_mapping[0].tolist(), labels[0].tolist()):
  if word_id is not None and offset[0] == 0:
    # print(f"Node: {nodes[0][word_id]}")
    # print(f"Predicted: {id2label[pred_id]}")
    # print(f"Ground truth: {id2label[label_id]}")
    # print("----------")
    results["Node"].append(nodes[0][word_id])
    results["Predicted"].append(id2label[pred_id])
    results["Ground truth"].append(id2label[label_id])

Let's pretty print the results as a Pandas dataframe:

In [ ]:
import pandas as pd

pd.DataFrame.from_dict(results).head()